In [1]:
import langchain

from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel, chain

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Together

from langchain.prompts.prompt import PromptTemplate
from operator import itemgetter

import os
import config
import json
import datetime
from io import StringIO

In [2]:
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = 'xxx'
os.environ['GROQ_API_KEY'] = 'xxx'

In [14]:
embeddings = HuggingFaceEmbeddings(model_name='hkunlp/instructor-large', # somehow only this works and the default HuggingFaceEmbeddings() 
                                   # i.e. sentence-transformers/all-mpnet-base-v2 and 'sentence-transformers/all-MiniLM-L6-v2' don't 
                                   # return any for retriever.invoke
                                           model_kwargs={'device': 'cpu', }) # cuda, cpu

db = FAISS.load_local('repo.db2', embeddings, allow_dangerous_deserialization=True)

retriever = db.as_retriever(
                search_type="similarity_score_threshold", # mmr # similarity_score_threshold
                search_kwargs={"k": 3, "score_threshold": 0.5}
)

In [35]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

# _template = """
# You're an AI assistant trained on the content of a web site.

# Given an optional Chat History and a New Question, rephrase the new question to be a standalone question. No pre-amble.

# If the New Question is a general greeting, set the standalone question to be the same as the New Question. No pre-amble.

# Chat History:
# {chat_history}

# New Question: {question}

# Standalone question:
# """

# CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

# search_template = """
# You are a helpful, respectful and honest assistant.
# Given a user question, your task is to enrich the user question with more relevant information.
# You can also summarize the chat history to add relevant context to the user question.
# Do not ask any follow up question to the users.

# Chat history:
# {chat_history}
# User question:
# {question}
# """
# SEARCH_PROMPT = PromptTemplate.from_template(search_template)

answer_template = """
You are a helpful, respectful and honest assistant.
If question is related to the context, you should ONLY use the provided CONTEXT to answer the question. DO NOT USE INFORMATION NOT IN THE CONTEXT.
If question is not related to the context, respond like a general AI assistant.

<CONTEXT>:
{context}
</CONTEXT>

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(answer_template)

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{source}: {page_content}")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    print(f"{doc_strings=}")
    return document_separator.join(doc_strings)

# _inputs = RunnableParallel(
#     standalone_question=RunnablePassthrough.assign(
#         chat_history=lambda x: get_buffer_string(x['chat_history'][-6:])
#     )
#     | SEARCH_PROMPT
#     | llm
#     | StrOutputParser(),
# )

# _context = {
#     "context": itemgetter("standalone_question") | retriever | _combine_documents,
#     "question": lambda x: x["standalone_question"],
# }

# chat_chain = _inputs | _context | ANSWER_PROMPT | llm

chat_chain = (ANSWER_PROMPT | llm)

@chain
def custom_chain(inputs):
    # refined_question = _inputs.invoke(inputs)
    
    # search_query = """
    # Question: {question}
    # {context_from_llm}
    # """

    # query_format = {"question": prompt, "context_from_llm": refined_question['standalone_question']}
    # search_query = search_query.format(**query_format)
    
    #docs = retriever.invoke(search_query)
    docs = retriever.invoke(inputs['question'])
    for doc in docs:
        if "url" in doc.metadata:
            doc.metadata['source'] = doc.metadata['url']

    context = _combine_documents(docs)

    answer = chat_chain.invoke({"context": context, "question": inputs['question']})
    print(f"{answer.content=}")
    return {'response': answer.content, 'source_document': docs}


In [15]:
docs_scores = db.similarity_search_with_score(query="What does LLM agents use?",
                                              k=4,
                                              score_threshold=0.2)
[doc for doc, score in docs_scores]


[]

In [13]:
HuggingFaceEmbeddings()

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [36]:
#prompt = "what is RAG Agent?"
prompt = "What does LLM agents use?"
answer = custom_chain.invoke(
    {
        "question": prompt,
        "chat_history": []
    }
)

response = answer['response']
response

doc_strings=['https://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.md: # LangChain <> Llama3 Cookbooks\n\n### `Agents`\n\nLLM agents use [planning, memory, and tools](https://lilianweng.github.io/posts/2023-06-23-agent/) to accomplish tasks. Here, we show how to build agents capable of [tool-calling](https://python.langchain.com/docs/integrations/chat/) using [LangGraph](https://python.langchain.com/docs/langgraph) with Llama 3.', 'https://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.md: Tool-calling agents with LangGraph use two nodes: (1) a LLM node decides which tool to invoke based upon the user input. It outputs the tool name and tool arguments to use based upon the input. (2) the tool name and arguments are passed to a tool node, which calls the tool with the specified arguments and returns the result back to the LLM.\n\n![Screenshot 2024-06-06 at 12 36 57 PM](https://github.com/rlancema

'According to the context, LLM agents use planning, memory, and tools to accomplish tasks.'

In [33]:
docs = retriever.invoke(prompt)    

In [34]:
docs

[Document(page_content='# LangChain <> Llama3 Cookbooks\n\n### `Agents`\n\nLLM agents use [planning, memory, and tools](https://lilianweng.github.io/posts/2023-06-23-agent/) to accomplish tasks. Here, we show how to build agents capable of [tool-calling](https://python.langchain.com/docs/integrations/chat/) using [LangGraph](https://python.langchain.com/docs/langgraph) with Llama 3.', metadata={'path': 'recipes/use_cases/agents/langchain/README.md', 'sha': '3040ff1be9deab4bcdd56f79d312a894b8ee46dd', 'source': 'https://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.md'}),
 Document(page_content='Tool-calling agents with LangGraph use two nodes: (1) a LLM node decides which tool to invoke based upon the user input. It outputs the tool name and tool arguments to use based upon the input. (2) the tool name and arguments are passed to a tool node, which calls the tool with the specified arguments and returns the result back to the LLM.\n\n![Scree

In [17]:
answer

{'response': 'According to the context, a RAG Agent is a custom Llama 3 powered agent that uses ideas from 3 papers and is built using LangGraph. It involves a sequence of steps (planning), memory to pass information between steps, and tool use for tasks such as retrieval.',
 'source_document': [Document(page_content='Our first notebook, `langgraph-tool-calling-agent`, shows how to build our agent mentioned above using LangGraph.\n\nSee this [video overview](https://youtu.be/j2OAeeujQ9M) for more detail on the design of this agent.\n\n--- \n\n### `RAG Agent`\n\nOur second notebook, `langgraph-rag-agent`, shows how to apply LangGraph to build a custom Llama 3 powered RAG agent that uses ideas from 3 papers:', metadata={'path': 'recipes/use_cases/agents/langchain/README.md', 'sha': '3040ff1be9deab4bcdd56f79d312a894b8ee46dd', 'source': 'https://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.md'}),
  Document(page_content='We implement each appr

In [18]:
docs = answer['source_document']

if len(docs) > 0:
    response += "\n> source:"

    for doc in docs:
        response += doc.metadata['source']
else:
   response = "Sorry, I can't find any relevant documents for your question. Please rephrase your question."


In [19]:
response

'According to the context, a RAG Agent is a custom Llama 3 powered agent that uses ideas from 3 papers and is built using LangGraph. It involves a sequence of steps (planning), memory to pass information between steps, and tool use for tasks such as retrieval.\n> source:https://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.mdhttps://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.mdhttps://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.md'

In [ ]:
prompt = "what is RAG Agent?"
#prompt = "What does LLM agents use?"
answer = custom_chain.invoke(
    {
        "question": prompt,
        "chat_history": []
    }
)

response = answer['response']
response

In [25]:
len(docs)

3

In [31]:
docs

[Document(page_content='Our first notebook, `langgraph-tool-calling-agent`, shows how to build our agent mentioned above using LangGraph.\n\nSee this [video overview](https://youtu.be/j2OAeeujQ9M) for more detail on the design of this agent.\n\n--- \n\n### `RAG Agent`\n\nOur second notebook, `langgraph-rag-agent`, shows how to apply LangGraph to build a custom Llama 3 powered RAG agent that uses ideas from 3 papers:', metadata={'path': 'recipes/use_cases/agents/langchain/README.md', 'sha': '3040ff1be9deab4bcdd56f79d312a894b8ee46dd', 'source': 'https://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.md'}),
 Document(page_content='We implement each approach as a control flow in LangGraph:\n- **Planning:** The sequence of RAG steps (e.g., retrieval, grading, and generation) that we want the agent to take.\n- **Memory:** All the RAG-related information (input question, retrieved documents, etc) that we want to pass between steps.\n- **Tool use:**

In [29]:
docs[2].metadata['url']=docs[2].metadata['source']

In [30]:
docs[2].metadata

{'path': 'recipes/use_cases/agents/langchain/README.md',
 'sha': '3040ff1be9deab4bcdd56f79d312a894b8ee46dd',
 'source': 'https://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.md',
 'url': 'https://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.md'}

In [40]:
from langchain_community.document_loaders import GitHubIssuesLoader
from langchain.document_loaders import GithubFileLoader

import os
from getpass import getpass

os.environ['GITHUB_PERSONAL_ACCESS_TOKEN'] = getpass()

 ········


In [41]:
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/github/
# Load github issues and PRs - API doc: https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.github.GitHubIssuesLoader.html
loader_issues = GitHubIssuesLoader(
    repo="meta-llama/llama-recipes",
    #include_prs=False,
    state='all'
)

docs_issues = loader_issues.load()

In [42]:
# load all markdowns files in a repo - API doc: https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.github.GithubFileLoader.html
loader_files = GithubFileLoader(
    repo="meta-llama/llama-recipes",
    github_api_url="https://api.github.com",
    file_filter=lambda file_path: file_path.endswith(
        ".md"
    ),  
)
docs_files = loader_files.load()

In [43]:
all_docs = docs_issues + docs_files

In [44]:
len(all_docs)

614

In [46]:
all_docs[-1]

Document(page_content='# Convert Hugging Face llama weights to official llama consolidated format\n\nThis is the reverse conversion for `convert_llama_weights_to_hf.py` script from the transformer package.\n\n## Step 0: Convert to consolidated format\n- Create an output directory for the converted weights, such as `test70B`.\n- Copy file params.json from the official llama download into that directory.\n- Run the conversion script. `model-path` can be a Hugging Face hub model or a local hf model directory.\n```\npython -m llama_recipes.tools.convert_hf_weights_to_llama --model-path meta-llama/Llama-2-70b-chat-hf --output-dir test70B --model-size 70B\n```\n\n## Step 1: Run inference\nCheckout the official llama inference [repo](https://github.com/facebookresearch/llama). Test using chat or text completion.\n```\ntorchrun --nproc_per_node 8 example_chat_completion.py --ckpt_dir ./test70B --tokenizer_path ${llama_2_dir}/tokenizer.model\n```\n\nFor validation, please compare the converted 

In [48]:
all_docs[-1].page_content

'# Convert Hugging Face llama weights to official llama consolidated format\n\nThis is the reverse conversion for `convert_llama_weights_to_hf.py` script from the transformer package.\n\n## Step 0: Convert to consolidated format\n- Create an output directory for the converted weights, such as `test70B`.\n- Copy file params.json from the official llama download into that directory.\n- Run the conversion script. `model-path` can be a Hugging Face hub model or a local hf model directory.\n```\npython -m llama_recipes.tools.convert_hf_weights_to_llama --model-path meta-llama/Llama-2-70b-chat-hf --output-dir test70B --model-size 70B\n```\n\n## Step 1: Run inference\nCheckout the official llama inference [repo](https://github.com/facebookresearch/llama). Test using chat or text completion.\n```\ntorchrun --nproc_per_node 8 example_chat_completion.py --ckpt_dir ./test70B --tokenizer_path ${llama_2_dir}/tokenizer.model\n```\n\nFor validation, please compare the converted weights with official 

In [49]:
all_docs[-1].metadata 
# a .md file's metadata

{'path': 'src/llama_recipes/utils/hf_llama_conversion/README.md',
 'sha': 'c02d9647e1668b044857795f18c54dd55b0d7046',
 'source': 'https://api.github.com/meta-llama/llama-recipes/blob/main/src/llama_recipes/utils/hf_llama_conversion/README.md'}

In [50]:
all_docs[0].metadata
# an issue/PR's metadata

{'url': 'https://github.com/meta-llama/llama-recipes/issues/574',
 'title': "What's the motivation of sorting dataset by length?",
 'creator': 'Ber666',
 'created_at': '2024-06-26T22:21:26Z',
 'comments': 1,
 'state': 'open',
 'labels': [],
 'assignee': None,
 'milestone': None,
 'locked': False,
 'number': 574,
 'is_pull_request': False}

In [53]:
# use this to diffrentiate an issue/PR from a .md file
'is_pull_request' in all_docs[0].metadata, 'is_pull_request' in all_docs[-1].metadata

(True, False)

In [58]:
if all_docs[0].metadata['is_pull_request']:
    print(1)
else:
    print(2)

2


In [61]:
prs, prs_closed, prs_open, issues, issues_closed, issues_open, mds = 0, 0, 0, 0, 0, 0, 0
for doc in all_docs:
    if 'is_pull_request' in doc.metadata:
        if doc.metadata['is_pull_request']:
            prs += 1            
            if doc.metadata['state'] == 'open':
                prs_open += 1
            else:
                prs_closed += 1
        else:
            issues += 1            
            if doc.metadata['state'] == 'open':
                issues_open += 1
            else:
                issues_closed += 1    
    else:
        mds += 1

In [62]:
prs, prs_closed, prs_open, issues, issues_closed, issues_open, mds

(279, 234, 45, 290, 204, 86, 45)

In [75]:
from langchain_core.tools import tool

@tool
def repo_statistics(stat):
    """Returns the statistics of a repo, including number of total issues, closed issues, open issues, total PRs, closed PRs, open PRs."""
    
    if "number_of_total_issues" == stat:
        return issues
    elif "number_of_closed_issues" == stat:
        return issues_closed
    elif "number_of_open_issues" == stat:
        return issues_open
    elif "number_of_total_prs" == stat:
        return prs
    elif "number_of_closed_prs" == stat:
        return prs_closed
    elif "number_of_open_prs" == stat:
        return prs_open
    else:
        return -1


In [85]:
@tool
def rag_query(prompt):
    """Search the repo for specific content, such as info about agents, fine-tuning"""
    
    answer = custom_chain.invoke(
        {
            "question": prompt,
            "chat_history": []
        }
    )
    
    response = answer['response']
    return(response)

In [112]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
First, use Thought to describe your thoughts about the question you have been asked, 
and generate Action in the following format:
Action: tool_name: tool_parameter

Then, return PAUSE.

Observation will be the result of running those tools. If the observation looks good as the answer, return it as the final answer.

Your available tool names with descrptions are as follows:
1. repo_statistics: useful for returning statistics about a repo, such as the number of total, closed or open issues or prs.
2. rag: useful for searching for specific content in a repo, such as info about agents, fine-tuning.

Examples to call the tools with tool name and parameter:
repo_statistics: number_of_closed_issues
repo_statistics: number_of_open_issues
repo_statistics: number_of_total_issues
repo_statistics: number_of_closed_prs
repo_statistics: number_of_open_prs
repo_statistics: number_of_total_prs
rag_query: original__question

""".strip()

In [113]:
messages = [
    ("system", system_prompt),
    ("human", "what is RAG agent"),
]
llm.invoke(messages)

AIMessage(content="Thought: I think the user is asking about the RAG agent, which is a tool used in the context of repositories. I'm going to use the rag tool to search for specific content in a repo related to agents, and fine-tuning.\n\nAction: rag_query: RAG agent\n\nPAUSE", response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 243, 'total_tokens': 305, 'completion_time': 0.177142857, 'prompt_time': 0.041989801, 'queue_time': None, 'total_time': 0.21913265799999998}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-99c8e78c-d62b-40be-af16-25ca67c0a63b-0')

In [114]:
messages = [
    ("system", system_prompt),
    ("human", "how many prs are closed?"),
]
llm.invoke(messages)

AIMessage(content='Thought: I need to find out the number of closed PRs in the repo. This information can be obtained by using the repo_statistics tool.\n\nAction: repo_statistics: number_of_closed_prs\n\nPAUSE', response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 245, 'total_tokens': 288, 'completion_time': 0.122857143, 'prompt_time': 0.065963963, 'queue_time': None, 'total_time': 0.18882110600000002}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_7ab5f7e105', 'finish_reason': 'stop', 'logprobs': None}, id='run-4e177e57-d583-463e-bdaa-1a51fabbed2e-0')

In [115]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        response = llm.invoke(self.messages)
        return response.content

In [135]:
import re

known_actions = {
    "repo_statistics": repo_statistics,
    "rag_query": rag_query
}
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

def query(question, max_turns=5):
    i = 0
    bot = Agent(system_prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))

            # key to make the agent process fully automated:
            # programtically call the external func with arguments, with the info returned by LLM
            observation = known_actions[action](action_input) 

            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return result

In [136]:
rslt = query("what is RAG agent")

Thought: I think the user is asking about the RAG agent, which is a tool used in the context of repositories. I'm going to use the rag tool to search for specific content in a repo related to agents, and fine-tuning.

Action: rag_query: RAG agent

PAUSE
 -- running rag_query RAG agent
doc_strings=['https://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.md: We will build from CRAG (blue, below) to Self-RAG (green) and finally to Adaptive RAG (red):\n\n![langgraph_rag_agent_](https://github.com/rlancemartin/llama-recipes/assets/122662504/ec4aa1cd-3c7e-4cd1-a1e7-7deddc4033a8)\n\n--- \n \n### `Local LangGraph RAG Agent`\n\nOur third notebook, `langgraph-rag-agent-local`, shows how to apply LangGraph to build advanced RAG agents using Llama 3 that run locally and reliably.', 'https://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.md: Our first notebook, `langgraph-tool-calling-agent`, shows how to buil

In [137]:
rslt

'Thought: It seems like the observation provides a clear answer to the question. The RAG agent is a custom Llama 3 powered RAG agent that uses ideas from 3 papers and is demonstrated in the langgraph-rag-agent notebook with a video overview available.\n\nAnswer: The RAG Agent is a custom Llama 3 powered RAG agent that uses ideas from 3 papers. It is demonstrated in the langgraph-rag-agent notebook, and there is also a video overview available that provides more detail on the design of this agent.'

In [118]:
query("What does LLM agents use?")

Thought: I'm thinking about what LLM agents use, and I'm considering the possibility that they might use some sort of repository or dataset to function. I want to know more about the repository they use.

Action: rag_query: LLM_agents

PAUSE
 -- running rag_query LLM_agents
doc_strings=['https://github.com/meta-llama/llama-recipes/issues/420: ```\r\n\r\nThe response from LLM was far from perfect. What am I missing here? I am open to any suggestions and help. Let me know if you need further information. Thank you in advance.', 'https://api.github.com/meta-llama/llama-recipes/blob/main/recipes/use_cases/agents/langchain/README.md: # LangChain <> Llama3 Cookbooks\n\n### `Agents`\n\nLLM agents use [planning, memory, and tools](https://lilianweng.github.io/posts/2023-06-23-agent/) to accomplish tasks. Here, we show how to build agents capable of [tool-calling](https://python.langchain.com/docs/integrations/chat/) using [LangGraph](https://python.langchain.com/docs/langgraph) with Llama 3.',

In [123]:
rslt

'Thought: It seems like the observation provides a clear answer to the question. The RAG agent is a custom Llama 3 powered RAG agent that uses ideas from 3 papers and is demonstrated in the langgraph-rag-agent notebook with a video overview available.\n\nAnswer: The RAG Agent is a custom Llama 3 powered RAG agent that uses ideas from 3 papers. It is demonstrated in the langgraph-rag-agent notebook, and there is also a video overview available that provides more detail on the design of this agent.'

In [130]:
rslt = query("how many prs are closed?")
rslt

Thought: I need to find out the number of closed PRs in the repo. This information can be obtained by using the repo_statistics tool.

Action: repo_statistics: number_of_closed_prs

PAUSE
 -- running repo_statistics number_of_closed_prs
Observation: 234
Observation: The number of closed PRs is 234.

Answer: 234


234

In [139]:
rslt = query("how many prs are closed? how many are open?")

Thought: I need to get the statistics about the pull requests in the repository to answer this question. I can use the repo_statistics tool to get the number of closed and open pull requests.

Action: repo_statistics: number_of_closed_prs
Action: repo_statistics: number_of_open_prs

PAUSE
 -- running repo_statistics number_of_closed_prs
Observation: 234
Thought: It seems like the observation only returned the number of closed PRs, which is 234. I still need to get the number of open PRs.

Action: repo_statistics: number_of_open_prs

PAUSE
 -- running repo_statistics number_of_open_prs
Observation: 45
Thought: Now I have the number of open PRs, which is 45. I can combine this with the previous observation to provide a complete answer.

Answer: There are 234 closed PRs and 45 open PRs.


In [140]:
rslt

'Thought: Now I have the number of open PRs, which is 45. I can combine this with the previous observation to provide a complete answer.\n\nAnswer: There are 234 closed PRs and 45 open PRs.'

In [ ]:
rslt.